In [ ]:
!pip install python-pptx python-docx
!pip install PyPDF2
!ollama pull nomic-embed-text
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
/bin/bash: line 1: ollama: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install notion-client

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from pptx import Presentation
from docx import Document
import io

# Authenticate and set up Drive service
auth.authenticate_user()
service = build('drive', 'v3')

# Specify the folder ID for the folder named "tes"
folder_id = '1v8XmlQE9tqfx74-5LUJrotVWxyZIPAYx'  # Replace with your actual folder ID

# Build the query to search for Google Docs, PDF, PPTX, and DOCX files in the specified folder
query = f"'{folder_id}' in parents and (mimeType='application/vnd.google-apps.document' or mimeType='application/pdf' or mimeType='application/vnd.openxmlformats-officedocument.presentationml.presentation' or mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')"

# List the files in the specific folder
results = service.files().list(q=query, pageSize=10, fields="nextPageToken, files(id, name, mimeType)").execute()
items = results.get('files', [])

if not items:
    print('No files found in the folder.')
else:
    print('Files in folder:')
    for item in items:
        print(f"{item['name']} ({item['id']}) - {item['mimeType']}")

# Function to download and extract text from files
def download_and_extract_text(file_id, mime_type):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()

    fh.seek(0)

    # Extract text based on MIME type
    if mime_type == 'application/vnd.google-apps.document':
        return fh.getvalue().decode('utf-8')

    elif mime_type == 'application/pdf':
        from PyPDF2 import PdfReader
        pdf_reader = PdfReader(fh)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.presentationml.presentation':  # PPTX
        pptx_file = io.BytesIO(fh.read())
        presentation = Presentation(pptx_file)
        text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text += shape.text + "\n"
        return text

    elif mime_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':  # DOCX
        docx_file = io.BytesIO(fh.read())
        document = Document(docx_file)
        text = ""
        for paragraph in document.paragraphs:
            text += paragraph.text + "\n"
        return text

    return None

# Extract text from files in the folder
documents_text = []
for item in items:
    file_id = item['id']
    mime_type = item['mimeType']
    text = download_and_extract_text(file_id, mime_type)
    documents_text.append((item['name'], text))

    print(f"Text from {item['name']}:\n{text[:100]}...\n")

Files in folder:
pembinaan jf sandiman dan MI 2022 final.pptx (15LCB3Qf6ejNofqehPxZcdx63vxxOiora) - application/vnd.openxmlformats-officedocument.presentationml.presentation
Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx (1tHTa1Y2ZcJMluhqEyIeqyvOGHGxSo4LK) - application/vnd.openxmlformats-officedocument.wordprocessingml.document
Paparan Keamanan Informasi (1).pdf (1mGQ7Dhuapr35mzhXZbhWV8DyKfy06VJv) - application/pdf
Text from pembinaan jf sandiman dan MI 2022 final.pptx:
”PEMBINAAN JF SANDIMAN DAN 
MANGGALA INFORMATIKA” 




















VISI INDONESIA 2045:
BERDAULA...

Text from Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx:
NOTULEN ACARA Enhancing Cybersecurity Awareness in Facing Cyber Threats

Hari 	: Kamis
Tanggal 	: 24...

Text from Paparan Keamanan Informasi (1).pdf:
Sosialisasi Keamanan
Informasi
Semarang, 21-24 Oktober 2024Dinas Komunikasi Informatika
Statistik da...



In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
from langchain.schema.document import Document
def load_documents():
    documents = []
    for name, text in documents_text:
        file_id = next(item['id'] for item in items if item['name'] == name)
        drive_link = f"https://drive.google.com/file/d/{file_id}/view"
        documents.append(Document(page_content=text, metadata={"source": name, "link": drive_link}))
        print(f"Loading document: {name}, link: {drive_link}")
    return documents


In [ ]:
documents = load_documents()
chunks = split_documents(documents)
print(chunks[0])

Loading document: pembinaan jf sandiman dan MI 2022 final.pptx, link: https://drive.google.com/file/d/15LCB3Qf6ejNofqehPxZcdx63vxxOiora/view
Loading document: Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx, link: https://drive.google.com/file/d/1tHTa1Y2ZcJMluhqEyIeqyvOGHGxSo4LK/view
Loading document: Paparan Keamanan Informasi (1).pdf, link: https://drive.google.com/file/d/1mGQ7Dhuapr35mzhXZbhWV8DyKfy06VJv/view
page_content='”PEMBINAAN JF SANDIMAN DAN 
MANGGALA INFORMATIKA” 




















VISI INDONESIA 2045:
BERDAULAT,  MAJU, ADIL DAN  MAKMUR
Revolusi Industri 4.0
Manusia Indonesia yang unggul,
berbudaya, serta menguasai ilmu pengetahuan dan teknologi
SDM  ASN  yang  profesional, bersih, kompeten,  netral dan  berintegritas sangat berperan dalam menentukan efektivitas pemerintah untuk mewujudkan visi pembangunan
Ekonomi yang maju dan
berkelanjutan
Pembangunan yang merata dan
inklusif
Negara yang demokratis, kuat,
dan bersih
Future of Government
(digital &

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  55232      0 --:--:-- --:--:-- --:--:-- 55058
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time

# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Let's use fly.io resources
#!OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 2755


In [ ]:
!ollama -v

ollama version is 0.5.7


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [ ]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [ ]:
from notion_client import Client

# Inisialisasi client dengan token API Notion
notion = Client(auth="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")  # Gantilah dengan token API kamu

# Fungsi untuk mengambil teks dari halaman Notion
def get_notion_page_text(page_id):
    blocks = notion.blocks.children.list(page_id)
    text_content = []
    for block in blocks['results']:
        if block['type'] == 'paragraph':
            paragraph_text = ''.join([text['text']['content'] for text in block['paragraph']['rich_text']])
            text_content.append(paragraph_text)
    return text_content

# Fungsi untuk mengambil data halaman dan URL Notion
def get_notion_page_data(page_id):
    page = notion.pages.retrieve(page_id)
    return page

# Ambil data halaman Notion
page_id = '17677ecf-d6c9-8063-a562-d3b7f10300de'
page_data = get_notion_page_data(page_id)

# Ambil data teks dari halaman Notion
notion_text_data = get_notion_page_text(page_id)

In [ ]:
from langchain.vectorstores.chroma import Chroma
CHROMA_PATH='chroma'

def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )
    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)
    # Periksa metadata sebelum menambahkan ke Chroma
    for chunk in chunks:
        print(f"Chunk ID: {chunk.metadata.get('id', 'No ID')} | Metadata: {chunk.metadata}")
    db.add_documents(chunks, ids=[chunk.metadata["id"] for chunk in chunks])
    db.persist()

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [ ]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 970aa74c0a90...   0% ▕▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕▏ 957 KB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  10% ▕▏  27 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  19% ▕▏  51 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  23% ▕▏  64 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  32% ▕▏  88 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  40% ▕▏ 110 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  46% ▕▏ 126 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  55% ▕▏ 151 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  62% ▕▏ 171 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  66% ▕▏ 180 MB/274 MB                  pulling manifest 
pulling 970aa74

In [ ]:
# Fungsi untuk menambahkan data Notion ke Chroma
def notion_to_chroma(notion_text_data, page_data, embedding_function):
    # Konversi data Notion menjadi format Document
    documents = []
    for i, text in enumerate(notion_text_data):
        metadata = {
            "content": text,
            "source": page_data['url'],  # Menyertakan URL halaman sebagai sumber
            "page": i + 1  # Menyertakan nomor halaman
        }
        documents.append(Document(page_content=text, metadata=metadata))

    # Tambahkan ke Chroma
    add_to_chroma_notion(documents, embedding_function)

# Fungsi untuk menambahkan dokumen ke Chroma
def add_to_chroma_notion(chunks, embedding_function):
    # Load the existing database.
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Kalkulasi ID untuk setiap chunk
    chunks_with_ids = calculate_chunk_ids_notion(chunks)

    # Tambahkan dokumen baru ke database
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])

    new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]
    if new_chunks:
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

# Fungsi untuk menghitung ID unik untuk setiap chunk
def calculate_chunk_ids_notion(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

# Gunakan data dari Notion untuk menambahkan ke Chroma
embedding_function_notion = get_embedding_function()
notion_to_chroma(notion_text_data, page_data, embedding_function_notion)

<ipython-input-11-35cf5b50dfff>:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")
<ipython-input-15-873a1b957146>:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)


👉 Adding new documents: 6


<ipython-input-15-873a1b957146>:33: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [ ]:
# Create (or update) the data store.
documents = load_documents()
chunks = split_documents(documents)
add_to_chroma(chunks)

Loading document: pembinaan jf sandiman dan MI 2022 final.pptx, link: https://drive.google.com/file/d/15LCB3Qf6ejNofqehPxZcdx63vxxOiora/view
Loading document: Notulen Enhancing Cybersecurity Awareness in Facing Cyber Threats.docx, link: https://drive.google.com/file/d/1tHTa1Y2ZcJMluhqEyIeqyvOGHGxSo4LK/view
Loading document: Paparan Keamanan Informasi (1).pdf, link: https://drive.google.com/file/d/1mGQ7Dhuapr35mzhXZbhWV8DyKfy06VJv/view
Chunk ID: pembinaan jf sandiman dan MI 2022 final.pptx:None:0 | Metadata: {'source': 'pembinaan jf sandiman dan MI 2022 final.pptx', 'link': 'https://drive.google.com/file/d/15LCB3Qf6ejNofqehPxZcdx63vxxOiora/view', 'id': 'pembinaan jf sandiman dan MI 2022 final.pptx:None:0'}
Chunk ID: pembinaan jf sandiman dan MI 2022 final.pptx:None:1 | Metadata: {'source': 'pembinaan jf sandiman dan MI 2022 final.pptx', 'link': 'https://drive.google.com/file/d/15LCB3Qf6ejNofqehPxZcdx63vxxOiora/view', 'id': 'pembinaan jf sandiman dan MI 2022 final.pptx:None:1'}
Chunk ID:

In [ ]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())
all_docs = db.get(include=["metadatas", "documents"])

for meta, doc in zip(all_docs["metadatas"], all_docs["documents"]):
    print(f"Document: {doc[:50]} | Metadata: {meta}")

Document: Ir. H. Joko Widodo adalah Presiden ke-7 Republik I | Metadata: {'content': 'Ir. H. Joko Widodo adalah Presiden ke-7 Republik Indonesia yang mulai menjabat sejak 20 Oktober 2014. Lahir di Surakarta, Jawa Tengah, pada 21 Juni 1961, Joko Widodo pertama kali terjun ke pemerintahan sebagai Wali Kota Surakarta (Solo) pada 28 Juli 2005 hingga 1 Oktober 2012.', 'id': 'https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de:1:0', 'page': 1, 'source': 'https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de'}
Document: Selepas itu, Joko Widodo menjabat sebagai Gubernur | Metadata: {'content': 'Selepas itu, Joko Widodo menjabat sebagai Gubernur DKI Jakarta pada 15 Oktober 2012 sebelum terpilih sebagai Presiden Republik Indonesia pada Pemilihan Presiden (Pilpres) 2014. Saat Pilpres tersebut Joko Widodo terpilih bersama pasangannya, Jusuf Kalla.', 'id': 'https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de:2:0', 'page': 2, 'source': 'https://www.notion.so/TESTING-1

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Jawab pertanyaan berdasarkan konteks:

{context}

---

Jawab pertanyaan berdasarkan konteks diatas: {question}
"""

def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=3)

    # Combine the context with links
    context_with_links = "\n\n---\n\n".join(
        f"{doc.page_content}\n(Source: {doc.metadata.get('link', doc.metadata.get('source', 'No link available'))})"
        for doc, _score in results
    )

    # Prepare the prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_with_links, question=query_text)

    # Query the model
    model = Ollama(model="llama2")
    response_text = model.invoke(prompt)

    # Extract and display sources with links
    sources = [doc.metadata.get("link", doc.metadata.get("source", 'No link available')) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {', '.join(sources)}"
    print(formatted_response)
    return response_text


In [ ]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 8934d96d3f08...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   0% ▕▏  11 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   1% ▕▏  43 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   2% ▕▏  69 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 115 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   3% ▕▏ 133 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   4% ▕▏ 159 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   5% ▕▏ 190 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   5% ▕▏ 209 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   6% ▕▏ 243 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   7% ▕▏ 274 MB/3.8 GB                  pulling manifest 
pulling 8934d96d3f08...   8% ▕▏ 29

In [ ]:
query_text = "joko widodo presiden RI ke berapa?"
query_rag(query_text)

<ipython-input-18-53933e8c9cd1>:33: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="llama2")


Response: Berdasarkan informasi di atas, Joko Widodo menjabat sebagai Presiden Republik Indonesia untuk masa jabatan 2014-2019 dan kembali terpilih untuk masa jabatan 2019-2024. Jadi, Joko Widodo presiden RI ke-7 dan ke-8.
Sources: https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de, https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de, https://www.notion.so/TESTING-17677ecfd6c98063a562d3b7f10300de


'Berdasarkan informasi di atas, Joko Widodo menjabat sebagai Presiden Republik Indonesia untuk masa jabatan 2014-2019 dan kembali terpilih untuk masa jabatan 2019-2024. Jadi, Joko Widodo presiden RI ke-7 dan ke-8.'